<a href="https://colab.research.google.com/github/DesiPilla/DSPS_dPilla/blob/master/HW5/BuildingsEnergyOwners.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
# -- link google drive
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# -- useful imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
import scipy as sp
from scipy import stats

# data retrival
### LL84
get the Local Law 84 (ll84) data from the URL: https://data.cityofnewyork.us/api/views/8u86-bviy/rows.csv and read it in with Pandas

### Pluto
Download the pluto csv data and store it in your google drive. 

You can find the data here 
https://www1.nyc.gov/site/planning/data-maps/open-data/dwn-pluto-mappluto.page

IMPORTANT: store it in the path *drive/My Drive/dsps/ with the name pluto_18v2_1.csv*

This path and name have to be exact. This way the grader and I will be able to run your notebook without changing anything as long as we also have stored the data in path drive/My Drive/dsps/pluto_18v2_1.csv. This is part of the _reproducibility_ requirement.

In [0]:
ll84 = pd.read_csv('https://data.cityofnewyork.us/api/views/8u86-bviy/rows.csv')

In [50]:
mkdir "drive/My Drive/dsps"

mkdir: cannot create directory ‘drive/My Drive/dsps’: No such file or directory


In [187]:
cd "drive/My Drive/dsps"

[Errno 2] No such file or directory: 'drive/My Drive/dsps'
/content/drive/My Drive/dsps/drive/My Drive/dsps


In [184]:
!wget https://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/nyc_pluto_18v2_1_csv.zip

--2019-09-29 21:39:40--  https://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/nyc_pluto_18v2_1_csv.zip
Resolving www1.nyc.gov (www1.nyc.gov)... 104.66.107.226, 2a02:26f0:6b:292::1500, 2a02:26f0:6b:2b1::1500
Connecting to www1.nyc.gov (www1.nyc.gov)|104.66.107.226|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 82994022 (79M) [application/zip]
Saving to: ‘nyc_pluto_18v2_1_csv.zip’

nyc_pluto_18v2_1_cs 100%[===================>]  79.15M  15.8MB/s    in 4.8s    

2019-09-29 21:39:45 (16.5 MB/s) - ‘nyc_pluto_18v2_1_csv.zip’ saved [82994022/82994022]



In [186]:
!unzip nyc_pluto_18v2_1_csv.zip

Archive:  nyc_pluto_18v2_1_csv.zip
replace pluto_18v2_1.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: pluto_18v2_1.csv        
  inflating: PLUTODD18v2.1.pdf       
  inflating: PlutoReadme18v2.1.pdf   


In [52]:
ls

pluto_18v2_1.csv


In [188]:
pluto = pd.read_csv("pluto_18v2_1.csv")
pluto.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (16,17,18,20,22,77) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,borough,block,lot,cd,ct2010,cb2010,schooldist,council,zipcode,firecomp,policeprct,healtharea,sanitboro,sanitsub,address,zonedist1,zonedist2,zonedist3,zonedist4,overlay1,overlay2,spdist1,spdist2,spdist3,ltdheight,splitzone,bldgclass,landuse,easements,ownertype,ownername,lotarea,bldgarea,comarea,resarea,officearea,retailarea,garagearea,strgearea,factryarea,...,assesstot,exemptland,exempttot,yearbuilt,yearalter1,yearalter2,histdist,landmark,builtfar,residfar,commfar,facilfar,borocode,bbl,condono,tract2010,xcoord,ycoord,zonemap,zmcode,sanborn,taxmap,edesignum,appbbl,appdate,mappluto_f,plutomapid,version,sanitdistrict,healthcenterdistrict,firm07_flag,pfirm15_flag,rpaddate,dcasdate,zoningdate,landmkdate,basempdate,masdate,polidate,edesigdate
0,BX,5641.000,670.000,210.000,516.000,2.000,nan,13.000,nan,NaN,nan,nan,nan,,CITY ISLAND AVENUE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,T2,7.000,0.000,NaN,MINNY C INC.,53758.000,0.000,nan,nan,nan,nan,nan,nan,nan,...,59400.000,0.000,0.000,0.000,0.000,0.000,NaN,NaN,0.000,0.000,0.000,0.000,2.000,2056410670.000,0.000,516.000,1044510.000,246734.000,4d,NaN,218 068,21810,NaN,2056410670.000,1/5/2005,nan,4.000,18v2.1,nan,nan,1.000,1.000,12/6/2018,12/20/2018,12/21/2018,12/20/2018,12/21/2018,nan,nan,12/20/2018
1,QN,3935.000,3.000,407.000,929.000,1001.000,25.000,19.000,11356.000,E297,109.000,210.000,4.000,5D,8-03 117 STREET,R2A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,N,A1,1.000,0.000,NaN,"CHEN, ZHIBIN",10870.000,3130.000,0.000,3130.000,0.000,0.000,0.000,0.000,0.000,...,53083.000,0.000,0.000,1915.000,1995.000,1996.000,NaN,NaN,0.290,0.500,0.000,1.000,4.000,4039350003.000,0.000,929.000,1025679.000,227096.000,7b,NaN,405 037,42101,NaN,0.000,NaN,nan,1.000,18v2.1,7.000,43.000,nan,nan,12/6/2018,12/20/2018,12/21/2018,12/20/2018,12/21/2018,nan,nan,12/20/2018
2,BX,4205.000,41.000,211.000,296.000,2000.000,11.000,13.000,nan,Q061,49.000,1410.000,2.000,2E,SEMINOLE AVENUE,R4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,N,V0,11.000,0.000,NaN,EDWARD AND LORRAINE S,2369.000,0.000,nan,nan,nan,nan,nan,nan,nan,...,5361.000,0.000,0.000,0.000,0.000,0.000,NaN,NaN,0.000,0.900,0.000,2.000,2.000,2042050041.000,0.000,296.000,1026345.000,249922.000,4a,NaN,220 053,21509,NaN,0.000,NaN,nan,1.000,18v2.1,11.000,26.000,nan,nan,12/6/2018,12/20/2018,12/21/2018,12/20/2018,12/21/2018,nan,nan,12/20/2018
3,BX,2402.000,14.000,201.000,69.000,2005.000,7.000,17.000,nan,L055,40.000,3900.000,2.000,1B,EAST 156 STREET,R6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,N,G7,10.000,0.000,NaN,"156, LLC",3750.000,0.000,nan,nan,nan,nan,nan,nan,nan,...,81000.000,0.000,0.000,0.000,0.000,0.000,NaN,NaN,0.000,2.430,0.000,4.800,2.000,2024020014.000,0.000,69.000,1007336.000,238351.000,6a,NaN,209N080,20905,NaN,0.000,NaN,nan,1.000,18v2.1,1.000,23.000,nan,nan,12/6/2018,12/20/2018,12/21/2018,12/20/2018,12/21/2018,nan,nan,12/20/2018
4,SI,63.000,83.000,501.000,81.000,2006.000,31.000,49.000,10301.000,E155,120.000,400.000,5.000,2C,216 FRANKLIN AVENUE,R2,NaN,NaN,NaN,NaN,NaN,HS,NaN,nan,NaN,N,A1,1.000,0.000,NaN,DENNIS COTTER,3465.000,1590.000,0.000,1590.000,0.000,0.000,0.000,0.000,0.000,...,20044.000,1460.000,1460.000,1901.000,0.000,0.000,NaN,NaN,0.460,0.500,0.000,1.000,5.000,5000630083.000,0.000,81.000,958845.000,172482.000,21a,NaN,501 082,50102,NaN,0.000,NaN,nan,1.000,18v2.1,1.000,51.000,nan,nan,12/6/2018,12/20/2018,12/21/2018,12/20/2018,12/21/2018,nan,nan,12/20/2018


# data fusion

### select data
select the data for buildings _in the borough of Manhattan_ only from the pluto file (you can do it broadcasting!)

### merge
merge the pluto and ll84 dataframes. you will have to choose an appropriate column that both datasets share fo do so. The column may not have the same nbame, but should have the same _variable_.

In [82]:
pluto = pluto[pluto["borough"] == "MN"]
pluto.shape

(42897, 96)

In [173]:
joined = pd.merge(pluto, ll84, left_on = "address", right_on = "Address 1 (self-reported)")
joined.shape

(53, 156)

# data exploration and cleaning

### extact columns
extract the columns listed in the file "columns.txt" from the dataset.

convert the resulting dataset to numerical, converting non-numerical values to NaN. drop rows that contain invalid values and save the resulting dataframe (_reducted_ dataframe)

In [174]:
columns = pd.read_table("https://raw.githubusercontent.com/fedhere/DSPS/master/HW5/columns.txt", header = None)
columns = columns[0].values.tolist()
print(columns)


data = joined[columns]
data = data.convert_objects(convert_numeric = True)
reduced = data.dropna(thresh = 22)
reduced.shape

['DOF Gross Floor Area', 'Year Built', 'Number of Buildings - Self-reported', 'Occupancy', 'ENERGY STAR Score', 'Site EUI (kBtu/ft²)', 'Weather Normalized Site EUI (kBtu/ft²)', 'Weather Normalized Site Electricity Intensity (kWh/ft²)', 'Weather Normalized Site Natural Gas Intensity (therms/ft²)', 'Weather Normalized Source EUI (kBtu/ft²)', 'Electricity Use - Grid Purchase (kBtu)', 'Total GHG Emissions (Metric Tons CO2e)', 'Property GFA - Self-Reported (ft²)', 'Water Use (All Water Sources) (kgal)', 'Water Intensity (All Water Sources) (gal/ft²)', 'Source EUI (kBtu/ft²)', 'bldgarea', 'numbldgs', 'numfloors', 'unitsres', 'unitstotal', 'yearalter1']


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  import sys


(14, 22)

In [175]:
reduced.head()

,DOF Gross Floor Area,Year Built,Number of Buildings - Self-reported,Occupancy,ENERGY STAR Score,Site EUI (kBtu/ft²),Weather Normalized Site EUI (kBtu/ft²),Weather Normalized Site Electricity Intensity (kWh/ft²),Weather Normalized Site Natural Gas Intensity (therms/ft²),Weather Normalized Source EUI (kBtu/ft²),Electricity Use - Grid Purchase (kBtu),Total GHG Emissions (Metric Tons CO2e),Property GFA - Self-Reported (ft²),Water Use (All Water Sources) (kgal),Water Intensity (All Water Sources) (gal/ft²),Source EUI (kBtu/ft²),bldgarea,numbldgs,numfloors,unitsres,unitstotal,yearalter1
1,91212.000,1927,1,100,85.000,64.700,68.000,11.200,0.100,155.000,3537948.900,483.800,92577,25098.800,271.110,150.900,91212.000,1.000,19.000,0.000,56.000,0.000
5,63600.000,1920,1,100,35.000,75.100,78.100,4.800,0.600,116.400,1215886.700,333.600,71550,1506.600,21.060,114.300,63600.000,1.000,8.000,28.000,30.000,0.000
7,55665.000,1900,1,100,29.000,108.100,113.300,5.100,1.000,155.300,904619.500,328.100,50892,2386.600,46.890,150.600,55665.000,1.000,7.000,43.000,43.000,0.000
9,137500.000,1990,1,100,83.000,35.800,34.900,8.700,0.100,98.900,4428400.900,450.200,144375,9539.200,66.070,101.700,117020.000,1.000,24.000,163.000,164.000,0.000
18,85194.000,1912,1,100,2.000,91.200,93.000,14.600,0.400,201.500,4447334.400,609.800,89453,6556.700,73.300,199.700,68728.000,1.000,14.000,66.000,104.000,2006.000


### Assess impact of data wrangling
Describe the  datasets resulting from this data cleaning: 

- how many observations have you lost? 

- did any of the features (=columns) have inconsistent distribution between the full and reducted dataset? 

You can approach this question in a number of ways, and you have done this in the  homewors already (with a KS test, and by just looking at uncertainties are two examples of ways to do ut that you worked out in HW3 and HW4, the Z test might work if you can convince yourself that the full sample is the "population" - up to you! a quick google search would probably describe many more ways). Choose _2_ ways, describe what each does and why it is appropriate. Compare the results (indeed I did it in 4 ways and did not always get consistent results! each of these meathods measures a slightly different thing).



**My answer:**

We started with 42,897 observations when we first joined the data frames. After eliminating all NaN / non-numeric answers, we were left with only 14 observations.

In [182]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

rmeans = pd.Series(reduced.mean(), name="reduced mean")
rstds = pd.Series(reduced.std(), name="reduced std")
fmeans = pd.Series(data.mean(), name="full mean")
fstds = pd.Series(data.std(), name="full std")
stats = pd.concat([rmeans, rstds, fmeans, fstds], axis = 1)

stats["zscore"] = (stats["reduced mean"] - stats["full mean"]) / stats["full std"]
stats

,reduced mean,reduced std,full mean,full std,zscore
DOF Gross Floor Area,279756.786,454917.066,178966.000,254326.308,0.396
Year Built,1941.786,36.071,1938.113,29.991,0.122
Number of Buildings - Self-reported,1.000,0.000,0.981,0.137,0.137
Occupancy,90.357,26.780,94.528,19.982,-0.209
ENERGY STAR Score,56.143,29.690,59.395,32.022,-0.102
Site EUI (kBtu/ft²),79.771,24.552,94.560,84.587,-0.175
Weather Normalized Site EUI (kBtu/ft²),82.336,25.827,102.250,109.741,-0.181
Weather Normalized Site Electricity Intensity (kWh/ft²),10.321,6.202,9.597,7.291,0.099
Weather Normalized Site Natural Gas Intensity (therms/ft²),0.214,0.303,0.352,0.517,-0.266
Weather Normalized Source EUI (kBtu/ft²),160.000,54.229,175.447,147.392,-0.105


In [183]:
max(abs(stats['zscore']))

0.43247559014942266

**My discussion**

The means of all of our reduced columns were within 0.5 standard deviations of the non-data-wranggled (assumed to be the full population) means. This supports the claim that the reduced data is consistent with the unreduced data.

### plot WNEC
Regardless of what you found above, plot the distribution of the _log base 10_ of the weather-normalized site energy consumption (WNEC) for all values larger than 0.1 (such that log10(min) is -1) for the _entire dataset_ and for the _reducted one_: do you think inference on this feature would be affected by this way of dealing with missing data affects the result?



# 667 and extyra credit
plot a "scatter matrix" of the selected columns. Identify 4 promising "scaling laws" and plot them separately (a scatter plot of column1 vs column2). Does it make sense that there should be a scaling law between the 2 variables? 

which features should be strongly correlated or identical but are not? discuss

([this](https://stackoverflow.com/questions/26975089/making-the-labels-of-the-scatterplot-vertical-and-horizontal-in-pandas) stackoverflow may come in handy)

## all students
At last, from the original dataset remove only the columns that are NaN in the column "Weather Normalized Site EUI (kBtu/ft²)"


In [0]:
# -- drop buildings for which WN Site EUI is NaN
wnec = "Weather Normalized Site EUI (kBtu/ft²)"
...
ll84pl.shape

## Analysis: Owner investigation 
### Find owner of the single most energy consuming building

**Find owner of the single most energy consuming building**, as measured by the "Site Weather Normalized EUI per unit area ". 

**Find all the buildings that they own.**


show the weather-normalized site energy consumption and the owner name for the top 30 consuming buildings in Manhattan

In [0]:
print("Owner of the hisghest consuming building:...", 
      "They own ... buildings")

### Find average building consumption by owner 
For all owners that own more than one building, find the average consumption for buildings owned by the same owner. Your dataframe should have 2 columns called _Weather Normalized Site EUI (kBtu/ft²)_	and _counts_

**Who owns the most energy consuming buildings on average?**

**Who owns the most buildings**

# 667 and EC

Universities are big realtors in NYC. The may own by different names. Find all builgings owned by NYU and COLUMBIA. replace the ownename with NYU and COLUMBIA respectively and re-aggregate the data based on these names and again find the owner of the buildings that pon average consume the most, of the highest number of buildings, and that owns the most sq footage 